# Exercise 1 — Prompt Chaining for a Customer Support AI
This notebook demonstrates a **multi-step prompt chain** where each step uses the prior step’s output.

Rubric focus:
- Clear multi-step flow (classify → gather info → propose solution → escalation)
- Prompts with constraints + structured outputs
- Evidence of iteration (v1 → v2)
- Successful output captured


In [ ]:
!pip -q install openai

# ===== Helper: LLM wrapper (REAL or MOCK) =====
# This notebook runs in MOCK mode by default so you always get "successful output"
# even without an API key. If you have an OpenAI key, set it and REAL mode will run.

import os, json, textwrap, re, sys
from dataclasses import dataclass

MODE = os.getenv("LLM_MODE", "MOCK").upper()  # set to REAL to use OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

@dataclass
class LLMResponse:
    text: str

def llm_call(system_prompt: str, user_prompt: str, mock_text: str) -> LLMResponse:
    """
    Returns LLMResponse.text.
    REAL mode uses OpenAI if OPENAI_API_KEY is set.
    MOCK mode returns mock_text (provided per call).
    """
    if MODE == "REAL":
        if not OPENAI_API_KEY:
            raise RuntimeError("REAL mode requested but OPENAI_API_KEY is not set.")
        # OpenAI SDK (recommended) — works in Colab once you `pip install openai`
        from openai import OpenAI
        client = OpenAI()
        resp = client.chat.completions.create(
            model=os.getenv("OPENAI_MODEL", "gpt-4.1-mini"),
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.2,
        )
        return LLMResponse(resp.choices[0].message.content)
    else:
        return LLMResponse(mock_text)

print(f"✅ LLM wrapper ready. MODE={MODE}")


## Scenario input
We’ll simulate a customer ticket, then run the prompt chain.

In [ ]:

ticket = {
  "customer_name": "Jordan",
  "product": "AcmeCloud Pro",
  "issue_text": "My account got charged twice this month and I can’t download my invoice.",
  "plan": "Pro Annual",
  "account_email": "jordan@example.com",
  "priority": "normal"
}
ticket


## Step 1 (v1) — Issue classification (initial prompt)
**Goal:** classify the issue category + urgency.

**Note (iteration evidence):** v1 is a little vague (weak constraints). We’ll improve it in v2.

In [ ]:

system_v1 = "You are a helpful customer support assistant."
prompt_step1_v1 = f"""
Classify the customer's issue and say if it's urgent.

Ticket:
{json.dumps(ticket, indent=2)}
"""
mock_step1_v1 = "Category: Billing + Account
Urgency: medium
Reason: double charge affects payment; invoice download issue blocks records."
resp1_v1 = llm_call(system_v1, prompt_step1_v1, mock_step1_v1).text
print(resp1_v1)


## Step 1 (v2) — Improved classification prompt
Improvements:
- Require **JSON** output
- Add **allowed categories**
- Constrain tone (neutral) and forbid sensitive info requests

In [ ]:

system_v2 = "You are a customer support triage bot for a SaaS company. Be concise and professional."
prompt_step1_v2 = f"""
TASK: Classify the ticket into ONE category and urgency.

CATEGORIES (choose one):
- billing
- technical
- account_access
- product_question
- cancellation_refund
- other

URGENCY (choose one): low | medium | high
RULES:
- Output MUST be valid JSON with keys: category, urgency, rationale (<=25 words).
- Do NOT ask for passwords, payment card numbers, or SSNs.

Ticket:
{json.dumps(ticket, indent=2)}
"""
mock_step1_v2 = json.dumps({
  "category":"billing",
  "urgency":"medium",
  "rationale":"Customer reports double charge and missing invoice download; financial impact with administrative blocker."
}, indent=2)

step1 = json.loads(llm_call(system_v2, prompt_step1_v2, mock_step1_v2).text)
step1


## Step 2 — Gather missing info (uses Step 1 output)
If category is billing, ask for the *minimum* details needed to investigate.
Output format: bullet questions + one friendly closing line.

In [ ]:

prompt_step2 = f"""
You are continuing a support workflow.

INPUTS:
- ticket: {json.dumps(ticket)}
- classification: {json.dumps(step1)}

TASK:
Ask up to 4 clarifying questions ONLY if needed to resolve the issue.
Rules:
- If classification.category == "billing": ask for transaction date(s), invoice month, and proof of duplicate charge (last 4 digits of card OK).
- Never request full card numbers, passwords, or SSN.
- Tone: friendly, calm.
- Output as:
  Questions:
  - ...
  Close: ...

Return only that format.
"""
mock_step2 = """Questions:
- Which billing month shows the duplicate charge (e.g., February 2026)?
- What are the transaction date(s) and amounts for the two charges?
- Does the charge appear on your bank/credit statement as “AcmeCloud Pro”?
- When you try to download the invoice, what error message do you see (or does it just not load)?
Close: Once I have those details, I can locate the duplicate and help you retrieve the invoice.
"""
step2_questions = llm_call(system_v2, prompt_step2, mock_step2).text
print(step2_questions)


## Step 3 — Propose solution (uses Steps 1–2)
We’ll simulate the customer's replies, then generate a resolution plan.

In [ ]:

customer_replies = {
  "billing_month": "February 2026",
  "charges": [
    {"date":"2026-02-03","amount":199.00},
    {"date":"2026-02-03","amount":199.00}
  ],
  "statement_descriptor": "ACMECLOUD*PRO",
  "invoice_error": "Invoice download button spins forever"
}
customer_replies


In [ ]:

prompt_step3 = f"""
You are a support agent. Use the workflow context to propose a solution.

CONTEXT:
ticket: {json.dumps(ticket)}
classification: {json.dumps(step1)}
clarifying_questions: {step2_questions}
customer_replies: {json.dumps(customer_replies)}

TASK:
1) Provide a short diagnosis.
2) Provide a step-by-step resolution plan (3–6 steps).
3) Include an escalation rule (when to escalate + to whom).
Constraints:
- Do not blame the customer.
- Do not promise refunds you can’t authorize. Use conditional language.
- Keep under 180 words.
Format:
Diagnosis:
Plan:
- ...
Escalate if: ...
"""
mock_step3 = """Diagnosis:
Looks like a duplicate billing event on 2026-02-03 plus a separate invoice-download UI issue.

Plan:
- I’ll verify both $199 charges for February 2026 against your AcmeCloud Pro account (jordan@example.com).
- If both charges posted, I’ll initiate a duplicate-charge investigation and request a correction per billing policy (you’ll receive confirmation by email).
- In parallel, try invoice download in an incognito window or a different browser; if it still spins, I’ll generate a manual PDF invoice for February 2026.
- I’ll also log the “spins forever” behavior with our product team including your browser + approximate time of attempt.

Escalate if: charges can’t be matched to your account within 1 business day → escalate to Billing Ops; invoice issue persists after troubleshooting → escalate to Engineering Support.
"""
final_response = llm_call(system_v2, prompt_step3, mock_step3).text
print(final_response)


✅ **What to copy into write-up document for Exercise 1**
- Tools: Colab, OpenAI API (optional), Python
- Prompts: Step 1 v1 + Step 1 v2 + Step 2 + Step 3
- Iteration: explain why v2 is better (JSON, constraints)
- Output: paste the printed outputs above
